# 웹 검색

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",  # general/news/finance
    search_depth="basic",  # basic/advance
    # time_range="day",  # 데이터 최신 기준, None/day/week/month/year
    # include_domains=None,
    # exclude_domains=None  # 특정 도메인 포함/제외
)

search_tool.invoke({'query': '서울고속버스터미널 근처에서, 제로페이가 되는 점심 먹을 만한 곳을 알려줘'})

{'query': '서울고속버스터미널 근처에서, 제로페이가 되는 점심 먹을 만한 곳을 알려줘',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.diningcode.com/list.dc?query=%EA%B3%A0%EC%86%8D%ED%84%B0%EB%AF%B8%EB%84%90%20%EC%A0%9C%EB%A1%9C%ED%8E%98%EC%9D%B4',
   'title': "'고속터미널 제로페이' 맛집 빅데이터 추천순위 Top3",
   'content': '고속터미널 제로페이맛집 (3곳) · 1. 이디야커피 서울고속버스터미널점 · 2. 이디야 반포지하상가점 · 3. 써브웨이 서울고속터미널점.',
   'score': 0.7961819,
   'raw_content': None},
  {'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EA%B3%A0%EC%86%8D%EB%B2%84%EC%8A%A4%ED%84%B0%EB%AF%B8%EB%84%90%20%EC%A0%9C%EB%A1%9C%ED%8E%98%EC%9D%B4',
   'title': "'강남고속버스터미널 제로페이' 맛집 빅데이터 추천순위 Top3",
   'content': '강남고속버스터미널 제로페이맛집 (3곳) · 1. 이디야커피 서울고속버스터미널점 · 2. 이디야 반포지하상가점 · 3. 써브웨이 서울고속터미널점.',
   'score': 0.79293,
   'raw_content': None},
  {'url': 'https://www.tripadvisor.co.kr/RestaurantsNear-g294197-d9579467-Express_Bus_Terminal-Seoul.html',
   'title': '서울 고속 버스 터미널 역 근처 맛집 Best 10 - Tripadvisor',
   'content

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template='''
너는 웹 검색결과를 요약해서, 질문에 맞게 잘 정리해서 좋은 답변을 주는 챗봇이야.
---
질문: {query}
---
검색결과: {search_results}
---
질문에 맞는 답을 검색결과 바탕으로 잘 정리해서 알려줘.
'''
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 RunnablePassthrough() 위치에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']}: {r['content']}' for r in x['results']]
            )
        )
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

chain.invoke('서울고속터미널에서 점심먹기 좋은 곳을 알려줘. 그런데, 제로페이나 비플페이가 가능한 곳만 알려줘')
# 단, 이 코드도 볼 일이 없다. agent 기능에 통합됨

"검색결과에서는 서울고속터미널에서 제로페이 또는 비플페이로 결제 가능한 점심 식사 장소에 대한 구체적인 정보는 찾기 어렵습니다. 하지만, 일반적으로 고속터미널 주변이나 터미널 내에서 제로페이와 비플페이 결제가 가능한 식당은 점차 늘어나고 있으니 다음과 같은 방법을 추천드립니다:\n\n1. 터미널 내 푸드코트 및 카페를 방문해보세요. 많은 곳이 무선 결제 수단을 도입하고 있으며, 제로페이 또는 비플페이 결제 가능 여부는 결제 시 확인하는 것이 좋습니다.\n2. 터미널 근처의 인기 있는 식당 검색 시 '제로페이' 또는 '비플페이' 지원 여부를 별도로 문의하거나, 내비게이션 또는 결제 앱에 제공된 정보를 참고하세요.\n3. 다이닝코드나 유튜브 리뷰 등에서는 제로페이와 비플페이 지원 식당 정보가 자주 업데이트되지 않으니, 방문 전에 해당 가게에 직접 문의하는 것이 가장 확실합니다.\n\n요약하면, 서울고속터미널 내 또는 근처에서 제로페이와 비플페이 결제 지원 식당은 가능성이 높으며, 방문 전 전화 문의 또는 결제 시 확인하시는 것을 추천드립니다."